In [1]:
#Data Exploration
import numpy as np
import pandas as pd
import xgboost ; import lightgbm

In [2]:
#read in data
train = pd.read_csv("train.csv")
members = pd.read_csv('members.csv')
songs = pd.read_csv('songs.csv')
song_extra_info = pd.read_csv('song_extra_info.csv')

In [3]:
rowin = train['msno'].nunique()
colin = train['song_id'].nunique()
train.shape[0] /(rowin * colin) #really really sparse data
colin

359966

In [4]:
#occurence preparation
trainme = train.merge(songs,on = 'song_id',how = 'left')[['msno','genre_ids','target']]
trainme = trainme[~pd.isnull(trainme.genre_ids)]
v = trainme.groupby(['msno','genre_ids','target']).size()
v = v.reset_index()
v.columns = ['msno','genre_ids','target','count']

In [5]:
# song_length become second or minute + seconds
round(songs.song_length.head()/1000%60)
#genres
from collections import defaultdict
song_gen = defaultdict(int)
for ids in songs.genre_ids:
    if not pd.isnull(ids):
        for sub in ids.split("|"):
            song_gen[sub] += 1

In [6]:
#compute the occurrence matrix based on genres
songmix = list(set(list(song_gen.keys()) + list(songs.genre_ids.unique())))
songmix = [(i,j) for i in songmix for j in range(2)]
dicti = dict(zip(songmix,range(len(songmix))))
v['genre_ids_map'] = v.apply(lambda row: dicti[(row['genre_ids'],row['target'])],axis = 1)
dictm = dict(zip(trainme.msno.unique(),range(trainme.msno.nunique())))
v['msno_ids_map'] = v['msno'].apply(lambda x: dictm[x])
#cooccurence matrix 
mat = np.zeros((v.msno.nunique(),len(songmix)))
for i in range(v.shape[0]):
    mat[v.msno_ids_map[i],v.genre_ids_map[i]] = v['count'][i]
colindex = []
for i,j in dicti.items(): #iteritems for python 3
    if not pd.isnull(i[0]) and "|" in i[0]:
        colindex.append(j)
        for ind in i[0].split("|"):
            mat[:,dicti[(ind,i[1])]] += mat[:,j]
#coln = [str(songmix[i][0])+str('_')+ str(songmix[i][1]) for i in range(len(songmix))]
mat = pd.DataFrame(mat,index = trainme.msno.unique(),columns = songmix)
mat = mat.iloc[:,[i for i in range(len(songmix)) if i not in colindex]]
mat = mat.loc[:,mat.sum(axis = 0) != 0]

In [14]:
#user similarities based on genre_preferences
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
mat_sparse = sparse.csr_matrix(mat)
u_simi= cosine_similarity(mat_sparse)

In [74]:
#for every song, calculate the probability based on user-similarities
train['sups'] = 0.0
train_gr = train.loc[train['msno'].isin(list(mat.index)),:]
train_gr = train.groupby('song_id').filter(lambda x: x.shape[0] > 1) #一个不值得分析
inddict = dict(zip(mat.index,range(mat.shape[0])))
train_gr['uind'] = train_gr['msno'].apply(lambda x: inddict[x])
train_gr['rating'] = train_gr['target'].apply(lambda x: 1 if x == 1 else -1)
for name,group in train_gr.groupby('song_id'):
    num = group.shape[0]
    for ind in group.index:
        sims = u_simi[group.uind[ind],group.uind]
        ra = (np.dot(sims,group.rating)-group.rating[ind])/(num-1)
        train.set_value(ind,'sups',ra)

In [3]:
#user similarity based on song listening?
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
rowin = train['msno'].unique()
colin = train['song_id'].unique()
spm = sparse.lil_matrix((len(rowin),len(colin)))
dictuser = dict(zip(rowin,range(len(rowin))))
dictsong = dict(zip(colin,range(len(colin))))
for ind in train.index:
    userp = dictuser[train.msno[ind]]
    songp = dictsong[train.song_id[ind]]
    spm[userp,songp] = (1 if train.target[ind] == 1 else -1)
u_simi_song= cosine_similarity(spm)
u_simi_song_r = np.round(u_simi_song,decimals = 3)

#what if we have other proportions instead of using all similar customers?[Unrealistic]
# train['usbs_0'] = 0.0; train['usbs_0.25'] = 0.0; train['usbs_0.5'] = 0.0; train['usbs_0.75'] = 0.0
# thres = [0,0.25,0.5,0.75]
# train['uind'] = train['msno'].apply(lambda x: dictuser[x])
# train['rating'] = train['target'].apply(lambda x: 1 if x == 1 else -1)
# for name,group in train.groupby('song_id'):
#     num = group.shape[0]
#     if num > 1:
#         for ind in group.index:
#             sims = u_simi_song_r[group.uind[ind],group.uind]
#             for thre in thres:
#                 sims = [sim if (sim >= thre or sim <= (-1*thre)) else 0 for sim in sims]
#                 ra = (np.dot(sims,group.rating)-group.rating[ind])/(num-1)
#                 train.set_value(ind,'usbs_'+str(thre),ra)



In [253]:
#try to calculate the score based on u_simi_song instead of u_simi
train['usbs'] = 0.0
train_gr = train.groupby('song_id').filter(lambda x: x.shape[0] > 1)
train_gr['uind'] = train_gr['msno'].apply(lambda x: dictuser[x])
train_gr['rating'] = train_gr['target'].apply(lambda x: 1 if x == 1 else -1)
for name,group in train_gr.groupby('song_id'):
    num = group.shape[0]
    if num > 1:
        for ind in group.index:
            sims = u_simi_song_r[group.uind[ind],group.uind]
            ra = (np.dot(sims,group.rating)-group.rating[ind])/(num-1)
            train.set_value(ind,'usbs',ra)
np.corrcoef(train.usbs,train.target)

In [87]:
#give each train data the score of the song prev and user prev, a song tend to be listened more and user tend to listen
tuf = train.groupby('msno')['target'].agg(['sum','count'])+[1,2] #Laplacian smoothing
tuf['freq'] = tuf.iloc[:,0]/tuf.iloc[:,1]
user_freqd = dict(zip(tuf.index,round(tuf.freq,3)))
train['user_freq'] = train['msno'].apply(lambda x: user_freqd[x])
tsf = train.groupby('song_id')['target'].agg(['sum','count'])+[1,2]
tsf['freq'] = tsf.iloc[:,0]/tsf.iloc[:,1]
song_freqd = dict(zip(tsf.index,round(tsf.freq,3)))
train['song_freq'] = train['song_id'].apply(lambda x: song_freqd[x])

array([[ 1.        ,  0.30058725],
       [ 0.30058725,  1.        ]])

In [123]:
#train usbs song with frequency 1, calculate score based on the similatiry of the song listened before/same artist/same genre
single_song = train.groupby('song_id').filter(lambda x: x.shape[0] == 1)
single_songd = train.loc[train.msno.isin(single_song.msno),:] #only 22个unique user 与unique song

,msno,song_id,source_system_tab,source_screen_name,source_type,target,uind,rating,usbs,user_freq,song_freq
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,1,1,0.0,0.742,0.667
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,1,1,0.0,0.742,0.667
25,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,WHEVGSHBtJBVy+HjH84qRtj0KcNtYd+2pUO4xHMAJXk=,my library,Online playlist more,online-playlist,0,6,-1,0.0,0.505,0.333
26,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,TLUDFNRpLCG/DmqLiXBc4Udypwucj4XmKtotIUO6nXM=,my library,Online playlist more,online-playlist,0,6,-1,0.0,0.505,0.333
35,RPLdWVYe6joZEnv9TzljsdCWMtKopKpRXk1hKAnQQCQ=,P6dllZw/ZnO7qcufxOKHAXGwlCvkmm4djJT/82d3ihA=,my library,Local playlist more,local-library,0,10,-1,0.0,0.332,0.333


In [139]:
single_song = train.groupby('song_id').filter(lambda x: x.shape[0] == 1)


In [97]:
#give the frequency of the song's artist be relistened by the user
train_artist = train.merge(songs,on = 'song_id',how = 'left')[['msno','song_id','artist_name','target']]
art_ana = train_artist.groupby(['msno','artist_name'])['target'].agg(['sum','count'])

In [109]:
from collections import defaultdict
artist_gen = defaultdict(int)
for ids in train_artist.artist_name:
    if not pd.isnull(ids):
        for sub in ids.split("|"):
            artist_gen[sub] += 1

0.7336053356351194

In [140]:
#mat decomposition, singular value decomposition
from sklearn.preprocessing import scale
#mat_std = scale(mat)
#u,s,v = np.linalg.svd(spm,full_matrices = 0)
from scipy import sparse
u,s,v= sparse.linalg.svds(spm,k=60)

In [134]:
s

array([ 235.76799303,  249.90794967,  455.59251689])

In [4]:
#How many days has they been active
members['rtime'] = members['registration_init_time'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['etime'] = members['expiration_date'].apply(
    lambda x : pd.to_datetime(str(x), format='%Y%m%d'))
members['numactivedays'] = (members['etime'] - members['rtime']).dt.days
min_day = members['rtime'].min()
members['day_id_res'] = (members['rtime'] - min_day).dt.days+1
members['day_id_exp'] = (members['etime'] - min_day).dt.days+1
members['r_year'] = members['rtime'].dt.year
members['r_month'] = members['rtime'].dt.month
members['r_day'] = members['rtime'].dt.day
members['e_year'] = members['etime'].dt.year
members['e_month'] = members['etime'].dt.month
members['e_day'] = members['etime'].dt.day

In [32]:
#cleaned age and cleaned_gender
members['is_male'] = members.gender == 'male'
members['is_female'] = members.gender == 'female'
members['age_mtrue'] = members.bd.apply(lambda x: 1 if x > 10 and x < 80 else 0)
weighted_age = round(members.bd[members.age_mtrue == 1].mean())
members['age_clean'] = members.bd.apply(lambda x: x if x >10 and x < 80 else weighted_age)
#Another method to clean data is based on their registration method, but all via 16 is 0

In [98]:
songs.language.value_counts()
song_index_zip =zip(songs.languages.unique())
songs['language_cate'] = songs.apply()

 52.0    1336694
-1.0      639467
 3.0      106295
 17.0      92518
 24.0      41744
 31.0      39201
 10.0      15482
 45.0      14435
 59.0       8098
 38.0       2385
Name: language, dtype: int64

In [145]:
v = pd.isnull(song_extra_info.isrc).sum()
song_extra_info[pd.isnull(song_extra_info.isrc)]

,song_id,name,isrc
15,uzWI7xZfL3gL2/B4ptZs0XfBuGC20ydak01SjhFuEtc=,如此美麗的妳,NaN
16,EXBuTr6J7UY6MDozwT/UDRVnmW0VGRVfeGBzrxVlX3k=,老鷹之歌,NaN
22,wSdWNC+BceQC4dBIgdNX7SGVXSFBhOCYTCWFtcvjmTw=,只要我長大,NaN
27,HCLMm7t3Wz4j3Bfc68yQ/kn32NQrM3OrfZmaxJIPoT8=,誰人愛我親像你,NaN
48,5Bodh/4SCpTLJiOSS5tHPXR7ra+Nk6FmrDHiHVEQNNI=,Shining Day,NaN
49,bNNgwWLzKX1Ox8n0sOlCsGEoxq9fEqSeTyHF+r0yA4o=,擔心,NaN
51,u3d60xvx0by7NOV/UZfvuGALrC5qRIeL1U1jNcBh3FM=,鴛鴦路,NaN
56,TdoS0gWqVxxNecd+124PnSbRkJxczG0t7bZxxkAspFU=,向前衝,NaN
60,g5wxmUw0UHCN9xnC0chMZDJBgriJr6/Db8AUwof611k=,男人的汗,NaN
65,ngL3kHTPKfDzxIAXDcbLiBcAByh2XOj5mm8AKbki474=,跟往事乾杯,NaN
